In [1]:
from dataclasses import dataclass
from typing import Optional
import matplotlib as mpl
import json
import sys
import numpy as np
import tables
import os
import matplotlib.pyplot as plt

In [2]:
from tables import File, Group, Table
path = "/home/zelenyy/data/thunderstorm/test_probability_dwyer/result.hdf5"

In [3]:
class WeightHistogramm:
    def __init__(self, bins_x, bins_y, data):
        self.bins_x = bins_x
        self.bins_y = bins_y
        self.data = data
    
    def get_weight(self, x, y) -> float:
        xc = np.argmax(self.bins_x>x) - 1
        yc = np.argmax(self.bins_y>y) - 1
        return self.data[xc,yc]
    
    @staticmethod
    def from_file(h5file: File, group: Group, tracking_table: Table):
        bins_x = h5file.get_node(group, "xbins").read()
        bins_y = h5file.get_node(group, "ybins").read()
        table = h5file.get_node(group, "histByEvent")
        data = table.read(table.nrows-1)['histogramm'][0]
        
        energy_min = np.min(bins_x)
        energy_max = np.max(bins_y)
        
        condition="(energy > {}) & (energy <= {}) & (particle == )".format(energy_min, energy_max)
        data_tracking = tracking_table.where(condition)
        
                
        return WeightHistogramm(bins_x, bins_y, data)

@dataclass
class HistDispatcher:
    gamma_low : WeightHistogramm
    gamma_high: WeightHistogramm
    electron_low: WeightHistogramm
    electron_high: WeightHistogramm
    
    def get_hist(self, particle) -> Optional[WeightHistogramm]:
        """
        PDGID
        electron = 11
        positron = -11
        gamma = 22
        :param particle: 
        :return: 
        """
        if particle["particle"] == 22:
            if particle["energy"] < 1:
                return self.gamma_low
            else:
                return self.gamma_high
        elif particle["particle"] == 11:
            if particle["energy"] < 1:
                return self.electron_low
            else:
                return self.electron_high
        else:
            return None
    @staticmethod
    def from_file(h5file, group):
        
        tracking_table = h5file.get_node(group, "TreeTracking")
        
        group_hist = h5file.get_node(group, "histDwyer2003")
        group_gamma_low = h5file.get_node(group_hist, "gamma_low")
        gamma_low = WeightHistogramm.from_file(h5file, group_gamma_low, tracking_table)
        
        group_gamma_high = h5file.get_node(group_hist, "gamma_high")
        gamma_high = WeightHistogramm.from_file(h5file, group_gamma_high, tracking_table)

        group_electron_low = h5file.get_node(group_hist, "electron_low")
        electron_low = WeightHistogramm.from_file(h5file, group_electron_low, tracking_table)        

        group_electron_high = h5file.get_node(group_hist, "electron_high")
        electron_high = WeightHistogramm.from_file(h5file, group_electron_high, tracking_table)
        
        return HistDispatcher(gamma_low, gamma_high, electron_low, electron_high)

In [27]:
with tables.open_file(path) as h5file:
    group =h5file.get_node(h5file.root, "sim0001")
    hist_dispatcher = HistDispatcher.from_file(h5file, group)
    

# %%

In [29]:
hist_dispatcher.gammaLowHist.data

array([[  7,  10,   3,  11,  11,  23,  29,  32,  58,  69,  72, 107, 162,
        195, 196, 157,  98,  43,  20,  11,   5,   2,   4,   0,   1,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0],
       [  4,   4,   6,   4,   3,  10,  19,   9,  35,  46,  46,  54,  83,
         83, 111,  72,  36,  20,   9,   5,   3,   2,   0,   0,   1,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0],
       [  3,   2,   4,   5,   7,   5,   7,  10,  16,  31,  31,  39,  55,
         55,  60,  49,  35,  20,  10,   3,   3,   0,   1,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0],
       [  2,   2,   1,   5,   4,   4,  10,   5,  12,  17,  21,  42,  46,
         40,  56,  30,  26,  11,   9,   5,   1,   1,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0],
       [  2,   0,   1,   0,   0,   4,   6,   7,  10,  11,  19,  25, 

In [6]:
with tables.open_file(path) as h5file:
    table =h5file.get_node(h5file.root, "sim0001/histDwyer2003/electron_high/xbins")
    print(repr(table.attrs))

/sim0001/histDwyer2003/electron_high/xbins._v_attrs (AttributeSet), 6 attributes:
   [CLASS := 'ARRAY',
    FLAVOR := 'numpy',
    TITLE := '',
    VERSION := '2.4',
    type := 'energy',
    unit := 'MeV']
